This jupyter notebook can be used to generate roadmap plots based on a given MRD. Place the MRD files in a subdirectory with the year, e.g. 2023, and then call it from this notebook.

Before changing this notebook, make sure the last working version is archived by executing the command ```jupyter nbconvert roadmap.ipynb --to html``` and moving the resulting html-file to the MRD-directory.

# Imports and basic types
Import all necessary packets and establish the basic types/classes used in this scipt. This is not relevant for the end user. To parametrize the script, please move on to the next section.

In [ ]:
import sys
import numpy as np
import pandas as pd

import datetime as dt
import os
import pathlib as pl

from matplotlib import pyplot as plt
from matplotlib import cm

In [ ]:
PACKAGE_ROOT = str(pl.Path(os.getcwd()).parent.parent)
if PACKAGE_ROOT not in sys.path:
    sys.path.append(PACKAGE_ROOT)
    
from projplan.focusareas import FocusAreas
from projplan.scenario import Scenario
from projplan.mrdplotting import MRDPlot

from projplan.helper import TimelineConstraints as TCS

# Configuration

Focus areas, i.e. the different task-groups the SW-ES team has to cover.

In [ ]:
FAs = {
        'admin':    FocusAreas('Admin',                                   1.00, False, '#dadada'), #gray
        'cusproj':  FocusAreas('Customer Project',                        1.50, False, '#009cdd'), #cyan
        'cussupp':  FocusAreas('Customer Support',                        0.50, False, '#28cfd7'), #cyan-green
        'overhead': FocusAreas('Project Overhead (Releasing etc.)',       1.00, False, '#6a8795'), #???
        'proj1':    FocusAreas('Proj1',                                   0.50, True,  '#004494'), #blue
        'proj2':    FocusAreas('Proj2',                                   0.50, True,  '#6ab023'), #green
}

MRD

In [ ]:
TIMELINE_START = '2024-01-01'
TIMELINE_END = '2045-12-31'
MRD_FILE = 'mrd.csv'

Plotting functions

In [ ]:
figureDir = pl.Path(os.getcwd()).joinpath('figures')
timelineEnd = dt.datetime(year=2029, month=12, day=31)
ylabelpos = -140
plotwidth = 20
plotheight = 5
tickMajorFreq = '2QE'
tickMinorFreq = 'QE'
legendOutside=True
boxaxespad=6
bbox_to_anchor=(0,1.3)
priorityLabels = ['', 'Basic Infrastructure', 'Basic Functionality', 'Basic Device', 'Advanced Device', 'Super Features']

# Roadmap plots

## Probable case

In [ ]:
scoBase05 = Scenario(
    name            = 'Base 0.5FTE',
    ftes            = 10,
    focusareas      = [fa for fa in FAs.values()],
    timelineStart   = TIMELINE_START,
    timelineEnd     = TIMELINE_END,
    mrdpath         = MRD_FILE,
)
scoBase05.workPerTask = 0.75
scoBase05.workDistribution = { 
                    'Proj1':            TCS([(dt.datetime(year=2029, month=1, day=1),None,0)]),
                    'Proj2':            TCS([
                                             (dt.datetime(year=2024, month=1, day=1),dt.datetime(year=2024, month=12, day=31),0.5),
                                             (dt.datetime(year=2025, month=1, day=1),dt.datetime(year=2025, month=12, day=31),1.0),
                                             (dt.datetime(year=2026, month=1, day=1),dt.datetime(year=2026, month=12, day=31),1.5),
                                             (dt.datetime(year=2027, month=1, day=1),dt.datetime(year=2027, month=12, day=31),2.0),
                                             (dt.datetime(year=2028, month=1, day=1),dt.datetime(year=2028, month=12, day=31),2.5),
                                           ])
                   }
scoBase05.scheduleTasks({idx: 'Proj2' for idx in scoBase05.timeline})
scoBase05Plot = MRDPlot(figureDir, [scoBase05])
scoBase05Plot.plot(  projectList=[platform for platform in scoBase05.platformFAs if platform.name in ['Proj2']]
                  , plotwidth=plotwidth, plotheight=plotheight
                  , legendOutside=legendOutside
                  , boxaxespad=boxaxespad, bbox_to_anchor=bbox_to_anchor
                  , tickMajorFreq=tickMajorFreq, tickMinorFreq=tickMinorFreq
                  , ylabelpos=ylabelpos, timelineEnd=timelineEnd
                  , priorityLabels=priorityLabels)
scoBase05Plot.save()

In [ ]:
scoBase20 = Scenario(
    name            = 'Base 2.0FTE',
    ftes            = 10,
    focusareas      = [fa for fa in FAs.values()],
    timelineStart   = TIMELINE_START,
    timelineEnd     = TIMELINE_END,
    mrdpath         = MRD_FILE,
)
scoBase20.workPerTask = 0.75
scoBase20.workDistribution = { 
                    'Proj1':            TCS([(dt.datetime(year=2026, month=1, day=1),None,0)]),
                    'Proj2':            TCS([
                                             (dt.datetime(year=2024, month=1, day=1),dt.datetime(year=2024, month=12, day=31),2.0),
                                             (dt.datetime(year=2025, month=1, day=1),dt.datetime(year=2025, month=12, day=31),2.5),
                                           ])
                   }
scoBase20.scheduleTasks({idx: 'Proj2' for idx in scoBase20.timeline})
scoBase20Plot = MRDPlot(figureDir, [scoBase20])
scoBase20Plot.plot(  projectList=[platform for platform in scoBase20.platformFAs if platform.name in ['Proj2']]
                  , plotwidth=plotwidth, plotheight=plotheight
                  , legendOutside=legendOutside
                  , boxaxespad=boxaxespad, bbox_to_anchor=bbox_to_anchor
                  , tickMajorFreq=tickMajorFreq, tickMinorFreq=tickMinorFreq
                  , ylabelpos=ylabelpos, timelineEnd=timelineEnd
                  , priorityLabels=priorityLabels)
scoBase20Plot.save()

In [ ]:
stacked = MRDPlot(figureDir, [scoBase05,scoBase20])
stacked.name = 'stacked_base'
stacked.plot(  projectList=[platform for platform in scoBase05.platformFAs if platform.name in ['Proj2']]
             , plotwidth=20, plotheight=4
             , legendOutside=False, boxaxespad=boxaxespad, bbox_to_anchor=bbox_to_anchor
             , tickMajorFreq=tickMajorFreq, tickMinorFreq=tickMinorFreq
             , ylabelpos=ylabelpos, timelineEnd=timelineEnd
             , priorityLabels=priorityLabels)
stacked.save()

## Added resources
Temporarily add 2 FTEs for Proj2

In [ ]:
addRes = Scenario(
    name            = 'Added Resources',
    ftes            = TCS([
                            (dt.datetime(year=2024, month=1, day=1),dt.datetime(year=2024, month=6, day=30),10),
                            (dt.datetime(year=2024, month=7, day=1),dt.datetime(year=2025, month=12, day=31),12),
                            (dt.datetime(year=2026, month=1, day=1),None,10)
                          ]),
    focusareas      = [fa for fa in FAs.values()],
    timelineStart   = TIMELINE_START,
    timelineEnd     = TIMELINE_END,
    mrdpath         = MRD_FILE,
)
addRes.workPerTask = 0.75
addRes.workDistribution = { 
                    'Proj1':          TCS([(dt.datetime(year=2027, month=1, day=1),None,0)]),
                    'Proj2':    TCS([
                                             (dt.datetime(year=2024, month=1, day=1),dt.datetime(year=2024, month=6, day=30),1.5),
                                             (dt.datetime(year=2024, month=7, day=1),dt.datetime(year=2024, month=12, day=31),3.5),
                                             (dt.datetime(year=2025, month=1, day=1),dt.datetime(year=2025, month=12, day=31),4.0),
                                             (dt.datetime(year=2026, month=1, day=1),dt.datetime(year=2026, month=12, day=31),2.5),
                                           ])
                   }
addRes.scheduleTasks({idx: 'Proj2' for idx in addRes.timeline})
scoBase2Plot = MRDPlot(figureDir, [addRes])
scoBase2Plot.plot( projectList=[platform for platform in addRes.platformFAs if platform.name in ['Proj2']]
                 , plotwidth=plotwidth, plotheight=plotheight
                 , legendOutside=legendOutside, boxaxespad=boxaxespad, bbox_to_anchor=bbox_to_anchor
                 , tickMajorFreq=tickMajorFreq, tickMinorFreq=tickMinorFreq
                 , ylabelpos=ylabelpos, timelineEnd=timelineEnd
                 , priorityLabels=priorityLabels)
scoBase2Plot.save()

# Velocity plot

### Estimated value curve

In [ ]:
velocityPlotScenario = scoBase05
extrapolateMax = 6000
finalIntervalLen = extrapolateMax * 0.3
transitionSteps = 5

def fitPoly(x, y, fitRange, deg=1):
    fit = np.polyfit(x, y, deg=deg)
    fitRangeList = range(fitRange[0], fitRange[1])
    return np.array([fitRangeList, np.poly1d(fit)(fitRangeList)]), fit

# Generate Data
velocity = {}
yOff = {'Proj1': 0, 'Proj2': len(velocityPlotScenario.mrd[velocityPlotScenario.mrd['Proj1-Effort'] == 0].index)}
for project in velocityPlotScenario.platformFAs:
    xVal = [0] + list(velocityPlotScenario.mrd[velocityPlotScenario.mrd[project.effortColumnName] != 0][project.effortColumnName].cumsum())
    yVal = np.array(range(len(xVal))) - yOff[project.name]
    velocity[project.name] = np.array([xVal,yVal])

extraVel = {}
productivity = {'Proj1': -0.5, 'Proj2': 0.4}    # Productivity change
phase1Max = max([round(velocity[project.name][0,-1]) for project in velocityPlotScenario.platformFAs])
for project in velocityPlotScenario.platformFAs:
    extraVel[project.name] = [None for _ in range(transitionSteps)]
    # Fit linear curve to estimated data
    extraVel[project.name][0], fit = fitPoly(velocity[project.name][0,:], velocity[project.name][1,:], [0,phase1Max], 1)
    
    transitionIntervall = (extrapolateMax - finalIntervalLen - extraVel[project.name][0][0,-1]) / transitionSteps
    productivityFactor = fit[0]
    for step in range(1,transitionSteps):
        if step == transitionSteps-1:
            transitionIntervall = finalIntervalLen
        # Calculate linear cuve splines to extrapolated data after transition
        productivityFactor = productivityFactor + ((fit[0] * productivity[project.name]) / transitionSteps)
        phase3offset = extraVel[project.name][step-1][1,-1] - productivityFactor * extraVel[project.name][step-1][0,-1]
        phase3xvals = np.arange(extraVel[project.name][step-1][0,-1], extrapolateMax if step == transitionSteps-1 else extraVel[project.name][step-1][0,-1]+transitionIntervall)
        extraVel[project.name][step] = np.array([phase3xvals, productivityFactor * phase3xvals + phase3offset])
        
# Merge extrapolation curves
extrapolatedCurves = {}
for project in velocityPlotScenario.platformFAs:
    concatCurves = extraVel[project.name][0]
    for step in range(1,transitionSteps):
        concatCurves = np.concatenate((concatCurves, extraVel[project.name][step]), axis=1)
    extrapolatedCurves[project.name] = np.array([range(int(concatCurves[0,:].max())), np.interp(range(int(concatCurves[0,:].max())), concatCurves[0,:], concatCurves[1,:])])


In [ ]:
#---- create figure ----

fwidth = 18.  # total width of the figure in inches
fheight = 3.5 # total height of the figure in inches

fig = plt.figure(figsize=(fwidth, fheight), dpi=100)

#---- define margins -> size in inches / figure dimension ----

left_margin  = 1.5 / fwidth
right_margin = 0.2 / fwidth
bottom_margin = 0.5 / fheight
top_margin = 0.25 / fheight

#---- create axes ----

# dimensions are calculated relative to the figure size

x = left_margin    # horiz. position of bottom-left corner
y = bottom_margin  # vert. position of bottom-left corner
w = 1 - (left_margin + right_margin) # width of axes
h = 1 - (bottom_margin + top_margin) # height of axes

ax = fig.add_axes([x, y, w, h])

#---- Define the Ylabel position ----

# Location are defined in dimension relative to the figure size  

xloc =  0.25 / fwidth 
yloc =  y + h / 2.  


ylims = [-max(list(yOff.values())), max([projCurve[-1][1,:].max() for projCurve in extraVel.values()])]

ax.set_xlim([0,extrapolateMax])
ax.set_xlabel('Estimated Invested Effort for Feature Development (Days)', fontsize=25)

ax.set_ylim(ylims)
ax.set_yticks([])
ax.set_ylabel('Completed\nFeatures', rotation=0, fontsize=25, verticalalignment='center',horizontalalignment='center')
ax.yaxis.set_label_coords(xloc, yloc, transform = fig.transFigure);

ax.hlines([0], 0, extrapolateMax, color=velocityPlotScenario.colordict['Proj1'], linestyles='dashed')
ax.text(extrapolateMax, -1, '\"Must-Have\" Platform Feature Baseline', fontsize=18, verticalalignment= "top", horizontalalignment="right")

for project in velocityPlotScenario.platformFAs:
    ax.plot(velocity[project.name][0,:], velocity[project.name][1,:], color=velocityPlotScenario.colordict[project.name], label=project.name, linewidth=3)
    ax.plot(extrapolatedCurves[project.name][0,:], extrapolatedCurves[project.name][1,:], color=velocityPlotScenario.colordict[project.name], linewidth=3, linestyle='dashdot')
        
# Find break even
interpolatedCurves = {}
for project in velocityPlotScenario.platformFAs:
    interpolatedCurves[project.name] = np.interp(range(int(max([curve[0,:].max() for curve in velocity.values()]))), velocity[project.name][0,:], velocity[project.name][1,:])
diffInterpolated = np.abs(interpolatedCurves['Proj2'] - interpolatedCurves['Proj1'])
breakeven = np.argmin(diffInterpolated[:-10])
ax.vlines(breakeven, ylims[0], ylims[1], color=velocityPlotScenario.colordict['Proj2'], linestyles='dashed')  
ax.set_xticks([breakeven], [str(breakeven)], fontsize=18);

ax.legend(loc='upper left', framealpha=1, fontsize=18);

#incompatible: fig.tight_layout()
fig.savefig(figureDir.joinpath('velocity.png'), dpi=300, bbox_inches = "tight")